<a href="https://colab.research.google.com/github/TerrorismAnalyticsBureau/TAB-AI/blob/main/TABAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

•    iyear, imonth, iday
•    country (the country code) and country_txt
•    region and region_txt
•    provstate and city
•    latitude and longitude (maybe)
•    attacktype1 and attacktype1_txt
•    targtype1 and targtype1_txt
•    targsubtype1 and targsubtype1_txt
•    target1 (the specific target by name, building or person), maybe
•    natity1 and natity1_txt (maybe later)
•    weaptype1 and weaptype1_txt (maybe)
•    nkill and nwound